# 分散表現を特徴量とした分類

- word2vecのベクトルの平均値をテキストのベクトルとして分類

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# Spacyの日本語モデルをロード
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')

# title から　'\u300012/4 10:35更新'　の部分を削除
feeds['title'] = feeds['title'].map(lambda x: x[:x.rfind('\u3000')] if x[-2:] == '更新' else x)

# text と summary を結合
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

### 教師データの作成

URLに基づいた教師ラベルの設定：

label 0
- business
- economy
- politics
- cat4 (nhk)
- cat5 (nhk)

label 1
- culture
- science
- sport
- cat2 (nhk)
- cat7 (nhk)

label 2
- 上記以外

In [ ]:
label_0 = ['business', 'economy', 'politics', 'cat4', 'cat5']
label_1 = ['culture', 'science', 'sport', 'cat2', 'cat7']

# label_0 の単語が url の中に含まれている：0
# label_1 の単語が url の中に含まれている：1
# それ以外：2
def get_label(url):
    url = url.lower()
    # map: label_0 の単語それぞれについて、url の中に含まれているか真偽を返す
    # any: mapの結果について論理和をとる
    if any(map(lambda x: x in url, label_0)):
        return 0
    if any(map(lambda x: x in url, label_1)):
        return 1
    else:
        return 2

# df['url'] について get_label を適用した結果を df['label'] として追加
df['label'] = df['url'].map(lambda x: get_label(x))
# 各ラベルの数を確認
df['label'].value_counts()

In [ ]:
# label 0, 1 を取り出す
df = df.query('label != 2')
# 数を確認
df['label'].value_counts()

### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### テキストのベクトル化

- word2vecのベクトルの平均値をテキストのベクトルとする

In [ ]:
# テキストのトークンのword2vecベクトルの平均値を計算
def text2vec(text):
    tokens = preprocess(text)
    total = nlp.vocab[' '].vector
    n = len(tokens)
    for t in tokens:
        vec = nlp.vocab[t].vector
        total = total + vec
    # ndarrayのベクトルをlistにして返す
    return (total / n).tolist()

# mapの結果はSeriesになるため、ndarrayからlistに変換することでlistのlistができ、
# それをDataFrameにする
vector = pd.DataFrame(df['text'].map(lambda x: text2vec(x)).values.tolist())
# 確認
vector

### 分類モデルの作成（学習）

- ナイーブベイズ分類器

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB

# 説明変数、目的変数
X = vector
Y = df.label

# ナイーブベイズ分類器
model = GaussianNB()

# 学習
model.fit(X, Y)
# 教師データについてモデルからの予測値を計算
Y_predict = model.predict(X)

# 特異度の計算
matrix = confusion_matrix(Y, Y_predict)
specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0])
# 精度
print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
    accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
    recall_score(Y, Y_predict), specificity))
# ROC, AUC
Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
plt.plot(fpr, tpr, label='AUC={:.3f}'.format(auc(fpr, tpr)))
plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.title('ROC曲線')
plt.legend()
plt.show()

### 交差検証法により予測精度を検証

- ナイーブベイズ分類器

In [ ]:
from sklearn.model_selection import cross_val_score

# 交差検証の実行
score = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
print('AUC={:.3f} (+/- {:.3f})'.format(score.mean(), score.std()))

### その他の分類器

- ロジスティック回帰
- 決定木
- ランダムフォレスト
- SVM、カーネルSVM